### Solving a set of linear Equations

In [1472]:
import numpy as np

In [1473]:
#Example function for Gauss-elimination with partial pivoting

In [1474]:
def GEPP(A, b, doPivot = True):
    '''
    Gaussian elimination with partial pivoting.
    
    input: A is an n x n numpy matrix
           b is an n x 1 numpy array
    output: x is the solution of Ax=b 
            with the entries permuted in 
            accordance with the pivoting 
            done by the algorithm
    post-condition: A and b have been modified.
    '''
    n = len(A)
    print(f'n={n}')
    if b.size != n:
        raise ValueError("Invalid argument: incompatible sizes between"+
                         "A and b.", b.size, n)
    # k represents the current pivot row. Since GE traverses the matrix in the 
    # upper right triangle, we also use k for indicating the k-th diagonal 
    # column index.
    
    # Elimination
    for k in range(n-1):
        if doPivot:
            # Pivot
            maxindex = abs(A[k:,k]).argmax() + k
            if A[maxindex, k] == 0:
                raise ValueError("Matrix is singular.")
            # Swap
            if maxindex != k:
                print('swapping rows')
                A[[k,maxindex]] = A[[maxindex, k]]
                b[[k,maxindex]] = b[[maxindex, k]]
        else:
            print('no-partial pivoting')
            if A[k, k] == 0:
                raise ValueError("Pivot element is zero. Try setting doPivot to True.")
        #Eliminate
        for row in range(k+1, n):
            multiplier = A[row,k]/A[k,k]
            A[row, k:] = A[row, k:] - multiplier*A[k, k:]
            b[row] = b[row] - multiplier*b[k]
    # Back Substitution
    print(A)
    x = np.zeros(n)
    for k in range(n-1, -1, -1):
        print(f'k={k}')
        
        x[k] = (b[k].item() - np.dot(A[k,k+1:n],x[k+1:n]))/A[k,k].item()
    return x

In [1475]:
#Example: 1
A = np.array([[1.0, 1.0, -1.0], [3.0, -1.0, 1.0], [1.0, -3.0, 2.0]])
b = np.array([[0.0], [4.0], [1.0]])

GEPP(A,b, doPivot=True)


n=3
swapping rows
swapping rows
[[ 3.         -1.          1.        ]
 [ 0.         -2.66666667  1.66666667]
 [ 0.          0.         -0.5       ]]
k=2
k=1
k=0


array([1., 2., 3.])

In [1476]:
A = np.array([[0.0007, 1.725], [0.4352, -5.433]])
b = np.array([[1.739], [3.271]])

GEPP(A,b, doPivot=True)

n=2
swapping rows
[[ 0.4352     -5.433     ]
 [ 0.          1.73373874]]
k=1
k=0


array([20.,  1.])

# <font face="Sans" color="blue"> LU Factorization</font>

We aim to decompose a matrix $A$ into $L$ and $U$, which represent _lower_ and _upper triangular matrix_ respectively. And $L$ must have $1$'s on its principal diagonal.
$$
A = LU
$$
For instance,
$$
A=\underbrace{\left[\begin{array}{cccc}
1 & 0 & 0 & 0 \\
* & 1 & 0 & 0 \\
* & * & 1 & 0 \\
* & * & * & 1
\end{array}\right]}_{L}\underbrace{\left[\begin{array}{cccc}
* & * & * & * \\
0 & * & * & * \\
0 & 0 & * & * \\
0 & 0 & 0 & *
\end{array}\right]}_{U}
$$

An example here:

$$
A =
\begin{bmatrix}
9 & 3 & 6\\3 & 4 & 6\\0 & 8 & 8\\
\end{bmatrix}
$$
Use Gaussian elimination to get U. Store the multipliers. They are elements of L

In [1477]:
import scipy as sp

In [1478]:
A = np.array([[1., 2., 3.], [6., 4., 5.], [8., 9., 7.]]); A
b = np.array([1., 2., 3.])
np.linalg.solve(A, b)

array([0.06666667, 0.06666667, 0.26666667])

In [1479]:
P, L, U = sp.linalg.lu(A)
# sp.linalg.lu?
print(P)
print(L)
print(U)

[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
[[ 1.          0.          0.        ]
 [ 0.75        1.          0.        ]
 [ 0.125      -0.31818182  1.        ]]
[[ 8.          9.          7.        ]
 [ 0.         -2.75       -0.25      ]
 [ 0.          0.          2.04545455]]


In [ ]:
def LU_Decomposition(A):
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy()
    for i in range(n):
        for j in range(i+1, n):
            factor = U[j, i]/U[i, i]
            U[j, :] -= factor*U[i, :]
            L[j, i] = factor
    return L, U
L, U = LU_Decomposition(A)
# print(L)
# print(U)
# np.dot?
def Forward_Substitution(L, b):
    y = np.zeros_like(b)
    for i in range(len(y)):
        y[i] = b[i] - np.dot(L[i, 0:i], y[0:i])
    return y
def Backward_Substitution(U, y):
    x = np.zeros_like(y)
    for i in range(len(x)-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, (i+1):], x[(i+1):]))/U[i, i]
    return x
# print(A, b)
# b = np.float64(b)
y = Forward_Substitution(L, b)
x = Backward_Substitution(U, y)
print(x)

[0.06666667 0.06666667 0.26666667]


Note that the SciPy ```lu``` function gives us $L$, $U$ and $P$. 

$P$ which is a **permutation matrix**. In general we may not need row swaps to convert $A$ into $U$. In some cases we need to switch/swap the rows, otherwise decomposition will not happen.

Thus Scipy uses $PLU$ decomposition to make the procedure always possible
$$
A  = PLU
$$

Solve the linear system：
$$
\begin{align}
3x_1-7x_2 -2x_3+2x_4&=-9\\
-3x_1+5x_2 +x_3 &=5\\
6x_1-4x_2 -5x_4&=7\\
-9x_1+5x_2 -5x_3+12x_4&=11\\
\end{align}
$$
In matrix form:
$$
\underbrace{\left[\begin{array}{rrrr}
3 & -7 & -2 & 2 \\
-3 & 5 & 1 & 0 \\
6 & -4 & 0 & -5 \\
-9 & 5 & -5 & 12
\end{array}\right]}_{A}
\left[\begin{array}{r}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{array}\right]
=
\underbrace{\left[\begin{array}{r}
-9 \\
5 \\
7 \\
11
\end{array}\right]}_{b}
$$
Perform $LU$ decomposition:

$$\underbrace{\left[\begin{array}{rrrr}
3 & -7 & -2 & 2 \\
-3 & 5 & 1 & 0 \\
6 & -4 & 0 & -5 \\
-9 & 5 & -5 & 12
\end{array}\right]}_{A}
=\underbrace{
\left[\begin{array}{rrrr}
1 & 0 & 0 & 0 \\
-1 & 1 & 0 & 0 \\
2 & -5 & 1 & 0 \\
-3 & 8 & 3 & 1
\end{array}\right]}_{L}\underbrace{\left[\begin{array}{rrrr}
3 & -7 & -2 & 2 \\
0 & -2 & -1 & 2 \\
0 & 0 & -1 & 1 \\
0 & 0 & 0 & -1
\end{array}\right]}_{U}$$

Replace $A$ by $LU$, we get $L(Ux) = b$, now solve this pair of equations

$$
Ly = b\\
Ux = y
$$

Construct augmented matrix $[L|b]$

$$
\underbrace{
\left[\begin{array}{rrrr}
1 & 0 & 0 & 0 \\
-1 & 1 & 0 & 0 \\
2 & -5 & 1 & 0 \\
-3 & 8 & 3 & 1
\end{array}\right]}_{L}
\underbrace{\left[\begin{array}{r}
y_1 \\
y_2 \\
y_3 \\
y_4
\end{array}\right]}_{y}
=
\underbrace{\left[\begin{array}{r}
-9 \\
5 \\
7 \\
11
\end{array}\right]}_{b}
$$

$$
\left[\begin{array}{r}
y_1 \\
y_2 \\
y_3 \\
y_4
\end{array}\right]=
\left[\begin{array}{r}
-9 \\
-4 \\
5 \\
1
\end{array}\right]
$$

Next we solve $Ux = y$ to get solution

### Why LU decomposition is better


| Feature                 | Gaussian Elimination                                 | LU Decomposition                                            |
| ----------------------- | ---------------------------------------------------- | ----------------------------------------------------------- |
| **Reusability**         | Must repeat elimination for every new ( \mathbf{b} ) | Reuse ( L ) and ( U ) for multiple ( \mathbf{b} )           |
| **Efficiency**          | ( O(n^3) ) for each right-hand side                  | ( O(n^3) ) once + ( O(n^2) ) per new ( \mathbf{b} )         |
| **Numerical stability** | May require pivoting manually                        | Can use **LU with pivoting (PA = LU)** for better stability |
| **Implementation**      | Conceptually simpler but repetitive                  | Standard in numerical libraries (NumPy, SciPy, LAPACK)      |
| **Storage**             | Doesn’t explicitly store triangular matrices         | Compactly stores both ( L ) and ( U )                       |


### When LU works and when it does not?

| Case                                       | LU possible?               | Comments                 |
| ------------------------------------------ | -------------------------- | ------------------------ |
| ( A ) full-rank and all leading minors ≠ 0 | Yes (no pivoting needed) | Simple LU                |
| ( A ) full-rank but needs row swaps        | Yes (with pivoting)      | Use ( PA = LU )          |
| ( A ) singular (rank deficient)            | No unique solution       | Use least squares or SVD |


In [1481]:
### Iterative methods

#### Gauss Jacobi Iteration Method

$$
x_i^{k+1} = \left(b_i -\sum_{j=1}^{i-1} a_{ij}x_j^k -\sum_{j=i+1}^{n} a_{ij}x_j^k\right)/a_{ii}
$$

In a condensed form:
$$
x_i^{k+1} = \left(b_i -\sum_{j=1, j\neq i}^n a_{ij}x_j^k\right)/a_{ii}
$$


How do I define $x_i^{k+1} - x_i^k$?

Let $e_i = x_i^{k+1} - x_i^k$

1. Measure of convergence is max($e_i$) < Tolerance (maximum error between old and new values of x will be compared with a user-defined Tolerance) $L_{\infty}$ norm

2. $L_2$ norm: $\sqrt{\sum_{i=1}^N e_i^2}$

##### Consider set of linear equations

$$
10 x_1 - x_2 + 2 x_3 = 6,
$$

$$
-x_1 + 11 x_2 -x_3 + 3 x_4 = 25,
$$

$$
2 x_1 - x_2 + 10x_3 -x_4 = -11,
$$

$$
-3x_2 -x_3 +8x_4 = 15
$$

In [1482]:
#create Ax = b, using numpy arrays

A = np.array([[10,-1,2,0],[-1,11,-1,3],[2,-1,10,-1],[0,-3,-1,8]])
b = np.array([[6,25,-11,15]])
b = np.transpose(b)

In [1483]:
A

array([[10, -1,  2,  0],
       [-1, 11, -1,  3],
       [ 2, -1, 10, -1],
       [ 0, -3, -1,  8]])

In [1484]:
b

array([[  6],
       [ 25],
       [-11],
       [ 15]])

In [1485]:
n = np.size(b)
n

4

In [1486]:
np.diagonal(A)

array([10, 11, 10,  8])

In [1487]:
z = np.reshape(np.diagonal(A),[n,1])
z

array([[10],
       [11],
       [10],
       [ 8]])

In [1488]:
D = np.diag(np.diagonal(A))
D

array([[10,  0,  0,  0],
       [ 0, 11,  0,  0],
       [ 0,  0, 10,  0],
       [ 0,  0,  0,  8]])

In [1489]:
LU = A - np.diag(np.diagonal(A))
LU

array([[ 0, -1,  2,  0],
       [-1,  0, -1,  3],
       [ 2, -1,  0, -1],
       [ 0, -3, -1,  0]])

In [1490]:
x_o = np.zeros_like(b)
x_o

array([[0],
       [0],
       [0],
       [0]])

In [1491]:
x = (b - np.dot(LU,x_o))/z
x

array([[ 0.6       ],
       [ 2.27272727],
       [-1.1       ],
       [ 1.875     ]])

In [1492]:
diff = np.linalg.norm(x-x_o, ord='fro')#'fro' = 2 for vectors
diff

np.float64(3.201704898362488)

In [1493]:
### Putting it all together in a python function
def myJacobi(A,b,max_iter,tol):
    n = np.size(A[0])
    b = np.reshape(b,[n,1])
    z = np.reshape(np.diagonal(A),[n,1])
    # i != j
    B = A - np.diag(np.diagonal(A))
    x_o = np.zeros_like(b)
    for i in range(max_iter):
        x = (b - np.dot(B,x_o))/z
        e = np.linalg.norm(x-x_o, ord='fro')
        # print(x,e)
        if e < tol:
            print(f'Iterations converged after {i} steps')
            break
        x_o = x
    return x

myJacobi(A,b,50,1e-6)

In [1494]:
x_50 = myJacobi(A, b, 50, 1e-6)
x_100 = myJacobi(A, b, 100, 1e-6)
print(x_50)
print(x_100)

Iterations converged after 12 steps
Iterations converged after 12 steps
[[ 0.9404958 ]
 [ 1.62975215]
 [-0.88760336]
 [ 2.37520638]]
[[ 0.9404958 ]
 [ 1.62975215]
 [-0.88760336]
 [ 2.37520638]]


In [1495]:
A@x_50-b

array([[-8.87167156e-07],
       [ 3.71939588e-07],
       [-5.39892024e-07],
       [-2.05826384e-06]])

### Gauss Seidel method

$$
x_i^{k+1} = \left(b_i - \sum_{j=1}^{i-1} a_{ij}x_j^{k+1} - \sum_{j=i+1}^n a_{ij}x_j^k\right)/a_{ii}
$$

In [1496]:
def mySeidel(A,b,max_iter,tol):
    n = np.size(A[0])
    b = np.reshape(b,[n,1])
    z = np.reshape(np.diagonal(A),[n,1])
    # i != j
    B = A - np.diag(np.diagonal(A))
    L = np.tril(B)
    U = np.triu(B)
    x = np.float64(np.zeros_like(b))
    # x_o = np.zeros_like(b)
    for i in range(max_iter):
        x_o = x.copy()
        for j in range(n):
            x[j] = (b[j] - np.dot(L[j,:], x) - np.dot(U[j,:], x_o))/z[j]
        # x = (b - np.dot(B,x_o))/z
        e = np.linalg.norm(x-x_o, ord='fro')
        # print(x,e)
        if e < tol:
            print(f'Iterations converged after {i} steps')
            break
    return x
x_50 = mySeidel(A, b, 50, 1e-12)
# x_100 = mySeidel(A, b, 100, 1e-12)
print(x_50)
# print(x_100)
A@x_50-b

Iterations converged after 13 steps
[[ 0.94049587]
 [ 1.62975207]
 [-0.88760331]
 [ 2.37520661]]


array([[ 4.82280882e-13],
       [-5.04485342e-13],
       [ 1.59872116e-13],
       [ 0.00000000e+00]])